**Don't change the grader**. It will be used to grade your code as is. Fix any errors in your library.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Your Code Imported Here

In [ ]:
from csc665 import features, metrics
import csc665.tree

# Prepare

In [ ]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import unittest as ut
import sklearn.tree

class Object(object): pass
class TC(ut.TestCase): pass
tc = TC()

In [ ]:
csv_df = pd.read_csv("data/Melbourne_housing_FULL.csv")

# Retest Library from Homework 1

It will be needed to use the Melbourne dataset. If you're seeing errors, 
please refer to the **sample implementation** of the library functions in **iLearn**.

## `preprocess_ver_1`

In [ ]:
_mel = features.Object()
_mel.X, _mel.y = features.preprocess_ver_1(csv_df, "Price")

In [ ]:
print(_mel.X.shape, _mel.y.shape)

assert _mel.X.shape[0] == 8887, "X.shape"
assert _mel.y.shape[0] == 8887, "y.shape"

## `train_test_split`

In [ ]:
_mel.X_train, _mel.X_test, _mel.y_train, _mel.y_test = features.train_test_split(
    _mel.X, _mel.y, test_size=0.2, shuffle=True, random_state=None)

In [ ]:
print(_mel.X.shape, _mel.X_train.shape, _mel.X_test.shape, _mel.y_train.shape, _mel.y_test.shape)

tc.assertEqual(_mel.X_test.shape[0], _mel.y_test.shape[0])
tc.assertEqual(_mel.X_train.shape[0], _mel.y_train.shape[0])

In [ ]:
if "DataFrame" in str(type(_mel.y_train)):
    _mel.y_train = _mel.y_train["Price"].values

if "DataFrame" in str(type(_mel.y_test)):
    _mel.y_test = _mel.y_test["Price"].values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100 , random_state=17)
%time _ = rf.fit(_mel.X_train, _mel.y_train)

## `MSE`

In [ ]:
_st = Object()
_act = Object()

_st.mse = metrics.mse(rf.predict(_mel.X_test), _mel.y_test)
_act.mse = mean_squared_error(_mel.y_test, rf.predict(_mel.X_test))

print(_st.mse, _act.mse)

tc.assertAlmostEqual(_st.mse, _act.mse, delta=1)

## $R^2$ - Test set

In [ ]:
_st.r2_test = metrics.rsq(rf.predict(_mel.X_test), _mel.y_test)
_act.r2_test = r2_score(_mel.y_test, rf.predict(_mel.X_test))

print(_st.r2_test, _act.r2_test)
tc.assertAlmostEqual(_st.r2_test, _act.r2_test, places=3)
tc.assertGreater(_st.r2_test, 0.6)

## $R^2$ - Train set

In [ ]:
_st.r2_train = metrics.rsq(rf.predict(_mel.X_train), _mel.y_train)
_act.r2_train = r2_score(_mel.y_train, rf.predict(_mel.X_train))

print(_st.r2_train, _act.r2_train)
tc.assertAlmostEqual(_st.r2_train, _act.r2_train, places=3)
tc.assertGreater(_st.r2_train, 0.6)

# Homework 2

In [ ]:
def compare_trees(sktree, our_tree, X_train, X_test, y_train, y_test):
    sktree.fit(X_train, y_train)
    our_tree.fit(X_train, y_train)
#     print(our_tree)
    
    expected = metrics.rsq(sktree.predict(X_train), y_train)
    actual = metrics.rsq(our_tree.predict(X_train), y_train)
    print("Train, sklearn, ours:", expected, actual)
    tc.assertAlmostEqual(expected, actual)
    
    expected = metrics.rsq(sktree.predict(X_test), y_test)
    actual = metrics.rsq(our_tree.predict(X_test), y_test)
    print("Test, sklearn, ours:", expected, actual)
    tc.assertLess(abs(expected - actual), 0.19)

## Income Dataset

In [ ]:
income_df = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Income2.csv")
income_df = income_df.drop("Unnamed: 0", axis=1)
_income = features.Object()
_income.X = income_df.drop("Income", axis=1)
_income.y = income_df["Income"].values

In [ ]:
RAND_STATE = 17

_income.X_train, _income.X_test, _income.y_train, _income.y_test = \
    features.train_test_split(
        _income.X, 
        _income.y, 
        test_size=0.2, 
        shuffle=True, 
        random_state=RAND_STATE)

In [ ]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=2, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=2, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

In [ ]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

In [ ]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

In [ ]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=5),
    csc665.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=5),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

In [ ]:
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=7),
    csc665.tree.DecisionTreeRegressor(max_depth=20, min_samples_leaf=7),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

## Melbourne Dataset

In [ ]:
_mel_sampled = features.Object()
_mel_sampled.X_train = _mel.X_train[:200]
_mel_sampled.y_train = _mel.y_train[:200]
_mel_sampled.X_test = _mel.X_test[:50]
_mel_sampled.y_test = _mel.y_test[:50]

In [ ]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=5, min_samples_leaf=1),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

In [ ]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=1),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

In [ ]:
%%time 
compare_trees(
    sklearn.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=7),
    csc665.tree.DecisionTreeRegressor(max_depth=100, min_samples_leaf=7),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)